In [2]:
import problem
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from merge_transformer import MergeTransformer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score


from six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
import os


In [3]:
def _encode_dates(X, drop=False):
    # With pandas < 1.0, we wil get a SettingWithCopyWarning
    # In our case, we will avoid this warning by triggering a copy
    # More information can be found at:
    # https://github.com/scikit-learn/scikit-learn/issues/16191
    X_encoded = X.copy()

    # Make sure that DateOfDeparture is of datetime format
    X_encoded.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
    # Encode the DateOfDeparture
    X_encoded.loc[:, 'year'] = X_encoded['DateOfDeparture'].dt.year
    X_encoded.loc[:, 'month'] = X_encoded['DateOfDeparture'].dt.month
    X_encoded.loc[:, 'day'] = X_encoded['DateOfDeparture'].dt.day
    X_encoded.loc[:, 'weekday'] = X_encoded['DateOfDeparture'].dt.weekday
    X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week
    X_encoded.loc[:, 'n_days'] = X_encoded['DateOfDeparture'].apply(
        lambda date: (date - pd.to_datetime("1970-01-01")).days)
    X_encoded.loc['day_nb'] = X_encoded['DateOfDeparture'].dt.dayofyear
    
    X_encoded['day_nb'] = X_encoded['DateOfDeparture'].dt.dayofyear

    X_encoded['leap_year_year'] = X_encoded['year'].apply(
        lambda x: True if x == 2012 else False)
    X_encoded['leap_year_month'] = X_encoded['month'].apply(
        lambda x: True if x > 2 else False)
    X_encoded.loc[:, 'leap_year'] = X_encoded.loc[:, 'leap_year_year'] & X_encoded.loc[:, 'leap_year_month']

    X_encoded['day_nb_leap'] = X_encoded.apply(lambda x: 
                x.day_nb - 1 if x.leap_year == True else x.day_nb, axis=1)
    X_encoded.drop(['leap_year_year', 'leap_year_month', 'leap_year', 'day_nb'], inplace=True, axis=1)
    X_encoded.rename({'day_nb_leap': 'day_nb'}, axis=1, inplace=True)
    
    if drop:
        X_encoded.drop('DateOfDeparture', inplace=True, axis=1)
        
    return X_encoded

## Get Train Test

In [4]:
X_train, y_train = problem.get_train_data('..')
X_test, y_test = problem.get_test_data('..')

X_train.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_train.loc[:, 'DateOfDeparture'])
X_test.loc[:, 'DateOfDeparture'] = pd.to_datetime(X_test.loc[:, 'DateOfDeparture'])

y_train = pd.DataFrame(y_train, columns=['Passengers'])
y_test = pd.DataFrame(y_test, columns=['Passengers'])

Xy_train = X_train.copy()
Xy_test = X_test.copy()
Xy_train['Passengers'] = y_train
Xy_test['Passengers'] = y_test

In [5]:
Xy_merged = pd.concat([Xy_train, Xy_test], ignore_index=True)

In [6]:
Xy_merged.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Passengers
0,2012-06-19,ORD,DFW,12.875000,9.812647,12.331296
1,2012-09-10,LAS,DEN,14.285714,9.466734,10.775182
2,2012-10-05,DEN,LAX,10.863636,9.035883,11.083177
3,2011-10-09,ATL,ORD,11.480000,7.990202,11.169268
4,2012-02-21,DEN,SFO,11.450000,9.517159,11.269364


In [7]:
date_encoder = FunctionTransformer(_encode_dates)
Xy_merged = date_encoder.fit_transform(Xy_merged)

<ipython-input-3-436ef0b91cdd>:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week


In [10]:
Xy_merged.isna().sum()


DateOfDeparture     1
Departure           1
Arrival             1
WeeksToDeparture    1
std_wtd             1
Passengers          1
year                1
month               1
day                 1
weekday             1
week                1
n_days              1
day_nb              1
dtype: int64

## Get Route Means

In [7]:
dep_list = Xy_merged['Departure'].unique()
arr_list = Xy_merged['Arrival'].unique()
routes = list(itertools.product(dep_list, arr_list))

# print(routes)

dict_df = {}
for route in routes:
    df1 = Xy_merged.loc[Xy_merged['Arrival'] == route[1]][Xy_merged['Departure'] == route[0]]
    df2 = Xy_merged.loc[Xy_merged['Departure'] == route[1]][Xy_merged['Arrival'] == route[0]]
    df = pd.concat([df1, df2])
    if not (df.empty):
        dict_df[route] = df

print(len(dict_df))
print(dep_list)

<ipython-input-7-d90bb533024d>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = Xy_merged.loc[Xy_merged['Arrival'] == route[1]][Xy_merged['Departure'] == route[0]]
<ipython-input-7-d90bb533024d>:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = Xy_merged.loc[Xy_merged['Departure'] == route[1]][Xy_merged['Arrival'] == route[0]]


128
['ORD' 'LAS' 'DEN' 'ATL' 'SFO' 'EWR' 'IAH' 'LAX' 'DFW' 'SEA' 'JFK' 'PHL'
 'MIA' 'DTW' 'BOS' 'MSP' 'CLT' 'MCO' 'PHX' 'LGA' nan]


In [8]:
df_routes = pd.DataFrame(routes, columns=['Departure', 'Arrival'])

In [9]:
df_routes.describe()

,Departure,Arrival
count,420,420
unique,20,20
top,LGA,EWR
freq,21,21


In [10]:
mean_list = []
std_list = []
routes_list = []
routes_str = []
weeks_to_dep = []

for k, v in dict_df.items():
    dep, arr = k
    std_list.append(v.Passengers.std())
    weeks_to_dep.append(v.WeeksToDeparture.mean())
    routes_list.append(k)
    routes_str.append(dep + "-" + arr)
    tuple_ = (dep, arr, v['Passengers'].mean())
    mean_list.append(tuple_)

df_means_routes = pd.DataFrame(mean_list, columns=['Departure', 'Arrival', 'route_mean'])

In [11]:
df_means_routes

,Departure,Arrival,route_mean
0,ORD,DFW,11.919010
1,ORD,DEN,10.737025
2,ORD,LAX,12.008992
3,ORD,SFO,11.719065
4,ORD,LAS,10.678536
...,...,...,...
123,LGA,CLT,10.773446
124,LGA,DTW,10.590409
125,LGA,MIA,10.972960
126,LGA,BOS,11.168998


In [12]:
# df_means_routes.to_csv('../data/routes_means.csv')

## Get Day Means

In [13]:
day_list = Xy_merged['weekday'].unique()
mean_list = []

for day in day_list:
    df = Xy_merged.loc[Xy_merged['weekday'] == day]
    tuple_ = (day, df['Passengers'].mean())
    mean_list.append(tuple_)

df_means_days = pd.DataFrame(mean_list, columns=['weekday', 'day_mean'])
df_means_days

,weekday,day_mean
0,1.0,11.080302
1,0.0,11.316519
2,4.0,11.221390
3,6.0,10.812631
4,3.0,11.358502
5,5.0,9.963934
6,2.0,11.222148
7,NaN,NaN


In [14]:
# df_means_days.to_csv('../data/weekdays_means.csv')

## Merge on day means

In [15]:
date_encoder = FunctionTransformer(_encode_dates)
X_train = date_encoder.fit_transform(X_train)
X_test = date_encoder.fit_transform(X_test)

merge_transform = MergeTransformer(
    X_ext=df_means_days, 
#     cols_to_rename={'Date': 'DateOfDeparture', 'Close': 'oil_stock_price', 'Volume': 'oil_stock_volume'},
    how='left',
    on=['weekday'])

X_train = merge_transform.fit_transform(X_train)

merge_transform = MergeTransformer(
    X_ext=df_means_days, 
#     cols_to_rename={'Date': 'DateOfDeparture', 'Close': 'oil_stock_price', 'Volume': 'oil_stock_volume'},
    how='left',
    on=['weekday'])

X_test = merge_transform.fit_transform(X_test)

X_train.head()

<ipython-input-2-436ef0b91cdd>:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week


,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,day_nb,day_mean
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012.0,6.0,19.0,1.0,25.0,15510.0,170.0,11.080302
1,2012-09-10,LAS,DEN,14.285714,9.466734,2012.0,9.0,10.0,0.0,37.0,15593.0,253.0,11.316519
2,2012-10-05,DEN,LAX,10.863636,9.035883,2012.0,10.0,5.0,4.0,40.0,15618.0,278.0,11.221390
3,2011-10-09,ATL,ORD,11.480000,7.990202,2011.0,10.0,9.0,6.0,40.0,15256.0,282.0,10.812631
4,2012-02-21,DEN,SFO,11.450000,9.517159,2012.0,2.0,21.0,1.0,8.0,15391.0,52.0,11.080302


## Get Month Means

In [16]:
month_list = Xy_merged['month'].unique()
mean_list = []

for month in month_list:
    df = Xy_merged.loc[Xy_merged['month'] == month]
    tuple_ = (month, df['Passengers'].mean())
    mean_list.append(tuple_)

df_means_month = pd.DataFrame(mean_list, columns=['month', 'month_mean'])
print(df_means_month.head())

merge_transform = MergeTransformer(
    X_ext=df_means_month, 
#     cols_to_rename={'Date': 'DateOfDeparture', 'Close': 'oil_stock_price', 'Volume': 'oil_stock_volume'},
    how='left',
    on=['month'])

X_train = merge_transform.fit_transform(X_train)

merge_transform = MergeTransformer(
    X_ext=df_means_month, 
#     cols_to_rename={'Date': 'DateOfDeparture', 'Close': 'oil_stock_price', 'Volume': 'oil_stock_volume'},
    how='left',
    on=['month'])

X_test = merge_transform.fit_transform(X_test)

X_train.head()

   month  month_mean
0    6.0   11.240980
1    9.0   11.163944
2   10.0   11.243521
3    2.0   10.926466
4    1.0   10.834784


,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,day_nb,day_mean,month_mean
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012.0,6.0,19.0,1.0,25.0,15510.0,170.0,11.080302,11.240980
1,2012-09-10,LAS,DEN,14.285714,9.466734,2012.0,9.0,10.0,0.0,37.0,15593.0,253.0,11.316519,11.163944
2,2012-10-05,DEN,LAX,10.863636,9.035883,2012.0,10.0,5.0,4.0,40.0,15618.0,278.0,11.221390,11.243521
3,2011-10-09,ATL,ORD,11.480000,7.990202,2011.0,10.0,9.0,6.0,40.0,15256.0,282.0,10.812631,11.243521
4,2012-02-21,DEN,SFO,11.450000,9.517159,2012.0,2.0,21.0,1.0,8.0,15391.0,52.0,11.080302,10.926466


In [17]:
# df_means_month.to_csv('../data/months_means.csv')

## Get Week Mean

In [18]:
week_list = Xy_merged['week'].unique()
mean_list = []

for week in week_list:
    df = Xy_merged.loc[Xy_merged['week'] == week]
    tuple_ = (week, df['Passengers'].mean())
    mean_list.append(tuple_)

df_means_week = pd.DataFrame(mean_list, columns=['week', 'week_mean'])
print(df_means_week.head())

merge_transform = MergeTransformer(
    X_ext=df_means_week, 
#     cols_to_rename={'Date': 'DateOfDeparture', 'Close': 'oil_stock_price', 'Volume': 'oil_stock_volume'},
    how='left',
    on=['week'])

X_train = merge_transform.fit_transform(X_train)

merge_transform = MergeTransformer(
    X_ext=df_means_week, 
#     cols_to_rename={'Date': 'DateOfDeparture', 'Close': 'oil_stock_price', 'Volume': 'oil_stock_volume'},
    how='left',
    on=['week'])

X_test = merge_transform.fit_transform(X_test)

X_train.head()

   week  week_mean
0  25.0  11.301741
1  37.0  11.282255
2  40.0  11.210159
3   8.0  10.950214
4   4.0  10.817967


,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,day_nb,day_mean,month_mean,week_mean
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012.0,6.0,19.0,1.0,25.0,15510.0,170.0,11.080302,11.240980,11.301741
1,2012-09-10,LAS,DEN,14.285714,9.466734,2012.0,9.0,10.0,0.0,37.0,15593.0,253.0,11.316519,11.163944,11.282255
2,2012-10-05,DEN,LAX,10.863636,9.035883,2012.0,10.0,5.0,4.0,40.0,15618.0,278.0,11.221390,11.243521,11.210159
3,2011-10-09,ATL,ORD,11.480000,7.990202,2011.0,10.0,9.0,6.0,40.0,15256.0,282.0,10.812631,11.243521,11.210159
4,2012-02-21,DEN,SFO,11.450000,9.517159,2012.0,2.0,21.0,1.0,8.0,15391.0,52.0,11.080302,10.926466,10.950214


In [19]:
# df_means_week.to_csv('../data/weeks_means.csv')

## Merge on Routes means

In [20]:
merge_transform = MergeTransformer(
    X_ext=df_means_routes, 
#     cols_to_rename={'Date': 'DateOfDeparture', 'Close': 'oil_stock_price', 'Volume': 'oil_stock_volume'},
    how='left',
    on=['Departure', 'Arrival'])

X_train = merge_transform.fit_transform(X_train)

merge_transform = MergeTransformer(
    X_ext=df_means_routes, 
#     cols_to_rename={'Date': 'DateOfDeparture', 'Close': 'oil_stock_price', 'Volume': 'oil_stock_volume'},
    how='left',
    on=['Departure', 'Arrival'])

X_test = merge_transform.fit_transform(X_test)

In [21]:
X_train

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,day_nb,day_mean,month_mean,week_mean,route_mean
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012.0,6.0,19.0,1.0,25.0,15510.0,170.0,11.080302,11.240980,11.301741,11.919010
1,2012-09-10,LAS,DEN,14.285714,9.466734,2012.0,9.0,10.0,0.0,37.0,15593.0,253.0,11.316519,11.163944,11.282255,10.451326
2,2012-10-05,DEN,LAX,10.863636,9.035883,2012.0,10.0,5.0,4.0,40.0,15618.0,278.0,11.221390,11.243521,11.210159,11.049831
3,2011-10-09,ATL,ORD,11.480000,7.990202,2011.0,10.0,9.0,6.0,40.0,15256.0,282.0,10.812631,11.243521,11.210159,11.300291
4,2012-02-21,DEN,SFO,11.450000,9.517159,2012.0,2.0,21.0,1.0,8.0,15391.0,52.0,11.080302,10.926466,10.950214,10.822171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8898,2012-09-25,DFW,ORD,12.772727,10.641034,2012.0,9.0,25.0,1.0,39.0,15608.0,268.0,11.080302,11.163944,11.254786,11.919010
8899,2012-01-19,SFO,LAS,11.047619,7.908705,2012.0,1.0,19.0,3.0,3.0,15358.0,19.0,11.358502,10.834784,10.926686,10.769626
8900,2013-02-03,ORD,PHL,6.076923,4.030334,2013.0,2.0,3.0,6.0,5.0,15739.0,34.0,10.812631,10.926466,10.829387,11.179790
8901,2011-11-26,DTW,ATL,9.526316,6.167733,2011.0,11.0,26.0,5.0,47.0,15304.0,330.0,9.963934,10.939110,10.183350,10.414953


In [22]:
X_test

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,day_nb,day_mean,month_mean,week_mean,route_mean
0,2012-05-21,LAS,ORD,12.000000,9.860938,2012.0,5.0,21.0,0.0,21.0,15481.0,141.0,11.316519,11.218335,11.091592,10.678536
1,2012-12-20,SFO,DEN,10.600000,9.954634,2012.0,12.0,20.0,3.0,51.0,15694.0,354.0,11.358502,10.604162,10.464457,10.822171
2,2012-11-01,LGA,DTW,11.950000,9.207977,2012.0,11.0,1.0,3.0,44.0,15645.0,305.0,11.358502,10.939110,10.878689,10.590409
3,2012-01-03,DEN,LAS,11.476190,9.352107,2012.0,1.0,3.0,1.0,1.0,15342.0,3.0,11.080302,10.834784,10.343277,10.451326
4,2012-11-19,LAX,ATL,13.444444,10.363892,2012.0,11.0,19.0,0.0,47.0,15663.0,323.0,11.316519,10.939110,10.183350,11.288425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,2012-09-01,ATL,DEN,11.000000,7.937254,2012.0,9.0,1.0,5.0,35.0,15584.0,244.0,9.963934,11.163944,10.791055,10.365239
2223,2012-08-10,ORD,EWR,9.476190,6.903760,2012.0,8.0,10.0,4.0,32.0,15562.0,222.0,11.221390,11.094082,11.097547,11.356683
2224,2012-07-03,ORD,IAH,10.250000,7.107261,2012.0,7.0,3.0,1.0,27.0,15524.0,184.0,11.080302,10.985364,10.325949,10.714472
2225,2012-02-25,DFW,PHL,6.727273,6.388911,2012.0,2.0,25.0,5.0,8.0,15395.0,56.0,9.963934,10.926466,10.950214,10.201987


In [23]:
# print("Testing RMSE: ", mean_squared_error(y_test, X_test['route_mean'], squared=False))

In [24]:
# print("Testing RMSE: ", mean_squared_error(y_train, X_train['route_mean'], squared=False))

## Get day mean

In [25]:
Xy_merged.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,Passengers,year,month,day,weekday,week,n_days,day_nb
0,2012-06-19,ORD,DFW,12.875000,9.812647,12.331296,2012.0,6.0,19.0,1.0,25.0,15510.0,170.0
1,2012-09-10,LAS,DEN,14.285714,9.466734,10.775182,2012.0,9.0,10.0,0.0,37.0,15593.0,253.0
2,2012-10-05,DEN,LAX,10.863636,9.035883,11.083177,2012.0,10.0,5.0,4.0,40.0,15618.0,278.0
3,2011-10-09,ATL,ORD,11.480000,7.990202,11.169268,2011.0,10.0,9.0,6.0,40.0,15256.0,282.0
4,2012-02-21,DEN,SFO,11.450000,9.517159,11.269364,2012.0,2.0,21.0,1.0,8.0,15391.0,52.0


In [26]:
# print(X_train['DateOfDeparture'][0].dayofyear)

# Xy_merged['day_nb'] = Xy_merged['DateOfDeparture'].dt.dayofyear

# Xy_merged['leap_year_year'] = Xy_merged['year'].apply(
#     lambda x: True if x == 2012 else False)
# Xy_merged['leap_year_month'] = Xy_merged['month'].apply(
#     lambda x: True if x > 2 else False)
# Xy_merged.loc[:, 'leap_year'] = Xy_merged.loc[:, 'leap_year_year'] & Xy_merged.loc[:, 'leap_year_month']

# Xy_merged['day_nb_leap'] = Xy_merged.apply(lambda x: 
#             x.day_nb - 1 if x.leap_year == True else x.day_nb, axis=1)
# Xy_merged.head()

In [27]:
day_nb_list = Xy_merged['day_nb'].unique()
mean_list = []

for day in day_nb_list:
    df = Xy_merged.loc[Xy_merged['day_nb'] == day]
    tuple_ = (day, df['Passengers'].mean())
    mean_list.append(tuple_)

df_means_day_nb = pd.DataFrame(mean_list, columns=['day_nb', 'day_nb_mean'])

merge_transform = MergeTransformer(
    X_ext=df_means_day_nb, 
    how='left',
    on=['day_nb'])

X_train = merge_transform.fit_transform(X_train)

merge_transform = MergeTransformer(
    X_ext=df_means_day_nb, 
    how='left',
    on=['day_nb'])

X_test = merge_transform.fit_transform(X_test)

X_train.head()


,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,year,month,day,weekday,week,n_days,day_nb,day_mean,month_mean,week_mean,route_mean,day_nb_mean
0,2012-06-19,ORD,DFW,12.875000,9.812647,2012.0,6.0,19.0,1.0,25.0,15510.0,170.0,11.080302,11.240980,11.301741,11.919010,11.203774
1,2012-09-10,LAS,DEN,14.285714,9.466734,2012.0,9.0,10.0,0.0,37.0,15593.0,253.0,11.316519,11.163944,11.282255,10.451326,10.768827
2,2012-10-05,DEN,LAX,10.863636,9.035883,2012.0,10.0,5.0,4.0,40.0,15618.0,278.0,11.221390,11.243521,11.210159,11.049831,11.528612
3,2011-10-09,ATL,ORD,11.480000,7.990202,2011.0,10.0,9.0,6.0,40.0,15256.0,282.0,10.812631,11.243521,11.210159,11.300291,11.201538
4,2012-02-21,DEN,SFO,11.450000,9.517159,2012.0,2.0,21.0,1.0,8.0,15391.0,52.0,11.080302,10.926466,10.950214,10.822171,11.330403


In [28]:
df_means_day_nb.to_csv('../data/day_nb_means.csv')

## Get Route/Weekday mean

In [17]:
# Xy_merged.dropna(inplace=True)
# weekdays = Xy_merged['weekday'].unique()
# dep_list = Xy_merged['Departure'].unique()
# arr_list = Xy_merged['Arrival'].unique()
# routes = list(itertools.product(dep_list, arr_list))

# mean_list = []

# routes_every_weekday = list(itertools.product(routes, weekdays))

# for route_weekday in routes_every_weekday:
#     route, weekday = route_weekday
#     dep, arr = route
#     df = Xy_merged.loc[(Xy_merged['weekday'] == weekday) & (Xy_merged['Departure'] == dep) & (Xy_merged['Arrival'] == arr)]
#     tuple_ = (weekday, dep, arr, df['Passengers'].mean())
#     mean_list.append(tuple_)

# df_means_weekday_routes = pd.DataFrame(mean_list, columns=['weekday', 'Departure', 'Arrival', 'weekday_route_mean'])
# df_means_weekday_routes

,weekday,Departure,Arrival,weekday_route_mean
0,1.0,ORD,DFW,12.017902
1,0.0,ORD,DFW,12.614183
2,4.0,ORD,DFW,12.267361
3,6.0,ORD,DFW,11.592206
4,3.0,ORD,DFW,12.482021
...,...,...,...,...
2795,4.0,LGA,IAH,NaN
2796,6.0,LGA,IAH,NaN
2797,3.0,LGA,IAH,NaN
2798,5.0,LGA,IAH,NaN


## Get Route/Month mean

In [18]:
Xy_merged.dropna(inplace=True)
months = Xy_merged['month'].unique()
dep_list = Xy_merged['Departure'].unique()
arr_list = Xy_merged['Arrival'].unique()
routes = list(itertools.product(dep_list, arr_list))

mean_list = []

routes_every_monthly = list(itertools.product(routes, months))

for route_month in routes_every_monthly:
    route, month = route_month
    dep, arr = route
    df = Xy_merged.loc[(Xy_merged['month'] == month) & (Xy_merged['Departure'] == dep) & (Xy_merged['Arrival'] == arr)]
    tuple_ = (month, dep, arr, df['Passengers'].mean())
    mean_list.append(tuple_)

df_means_weekday_routes = pd.DataFrame(mean_list, columns=['month', 'Departure', 'Arrival', 'weekday_route_mean'])
df_means_weekday_routes

,month,Departure,Arrival,weekday_route_mean
0,6.0,ORD,DFW,12.067778
1,9.0,ORD,DFW,12.180249
2,10.0,ORD,DFW,12.466226
3,2.0,ORD,DFW,12.452284
4,1.0,ORD,DFW,11.368380
...,...,...,...,...
4795,11.0,LGA,IAH,NaN
4796,3.0,LGA,IAH,NaN
4797,12.0,LGA,IAH,NaN
4798,8.0,LGA,IAH,NaN


In [20]:
df_means_weekday_routes.isna().sum()

month                    0
Departure                0
Arrival                  0
weekday_route_mean    3290
dtype: int64

,Departure,Arrival,month,WeeksToDeparture,std_wtd,Passengers,year,day,weekday,week,n_days,day_nb
0,ATL,BOS,1.0,10.970267,8.334175,10.448578,2012.750000,13.000000,3.750000,2.250000,15626.500000,13.000000
1,ATL,BOS,2.0,10.936442,9.213329,10.455955,2012.363636,14.818182,3.181818,6.909091,15517.909091,45.818182
2,ATL,BOS,3.0,9.894851,7.630884,11.128844,2012.222222,10.333333,3.222222,10.333333,15490.444444,69.333333
3,ATL,BOS,4.0,11.899793,9.204684,11.450565,2012.000000,21.000000,3.250000,16.250000,15451.000000,111.000000
4,ATL,BOS,5.0,11.038724,8.825557,11.274603,2012.000000,18.375000,3.500000,20.125000,15478.375000,138.375000
...,...,...,...,...,...,...,...,...,...,...,...,...
1505,SFO,SEA,8.0,13.753801,9.986754,11.045104,2012.000000,9.000000,4.400000,31.800000,15561.000000,221.000000
1506,SFO,SEA,9.0,12.718082,9.034077,11.383400,2011.615385,13.076923,2.846154,36.923077,15455.307692,256.076923
1507,SFO,SEA,10.0,12.753602,10.111313,11.361682,2011.600000,13.800000,2.200000,41.400000,15480.400000,286.800000
1508,SFO,SEA,11.0,9.802083,6.921058,11.452154,2011.500000,10.750000,3.000000,45.250000,15471.750000,314.750000


,Unnamed: 0,DateOfDeparture,AirPort,n_days,day_nb,oil_stock_price,oil_stock_volume,AAL_stock_price,AAL_stock_volume,SP_stock_price,...,week_mean,month_mean,day_nb_mean,route_mean,Arrival,distance,Total,Flights,Booths,Mean per flight
0,31104,01/09/2011,ATL,15218,244,88.93,282342.0,5.29,6677100.0,1204.420044,...,10.791055,11.163944,10.7111,11.554774,DFW,1177.006614,5061.0,51.0,244.0,99.24
1,31105,01/09/2011,ATL,15218,244,88.93,282342.0,5.29,6677100.0,1204.420044,...,10.791055,11.163944,10.7111,10.365239,DEN,1930.017732,803.0,7.0,31.0,114.71
2,31106,01/09/2011,ATL,15218,244,88.93,282342.0,5.29,6677100.0,1204.420044,...,10.791055,11.163944,10.7111,11.288425,LAX,3132.519497,22513.0,105.0,928.0,214.41
3,31107,01/09/2011,ATL,15218,244,88.93,282342.0,5.29,6677100.0,1204.420044,...,10.791055,11.163944,10.7111,11.300291,ORD,974.957144,12436.0,62.0,412.0,200.58
4,31108,01/09/2011,ATL,15218,244,88.93,282342.0,5.29,6677100.0,1204.420044,...,10.791055,11.163944,10.7111,10.462382,SFO,3442.220394,11567.0,49.0,518.0,236.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70651,101755,05/03/2013,LGA,15769,64,90.82,209874.0,14.05,7676100.0,1539.790039,...,10.798397,10.946436,10.6809,10.773446,CLT,875.196007,2583.0,17.0,80.0,151.94
70652,101756,05/03/2013,LGA,15769,64,90.82,209874.0,14.05,7676100.0,1539.790039,...,10.798397,10.946436,10.6809,10.590409,DTW,807.104383,2938.0,20.0,144.0,146.90
70653,101757,05/03/2013,LGA,15769,64,90.82,209874.0,14.05,7676100.0,1539.790039,...,10.798397,10.946436,10.6809,10.972960,MIA,1764.808838,21791.0,173.0,832.0,125.96
70654,101758,05/03/2013,LGA,15769,64,90.82,209874.0,14.05,7676100.0,1539.790039,...,10.798397,10.946436,10.6809,11.168998,BOS,297.068044,2501.0,22.0,129.0,113.68


In [36]:
ext_data = pd.read_csv('../data/external_data.csv')
ext_data

date_encoder = FunctionTransformer(_encode_dates)
ext_data = date_encoder.fit_transform(ext_data)

ext_data.drop('Unnamed: 0', axis=1, inplace=True)
print(ext_data.columns)

X_groupby = pd.DataFrame(Xy_merged.groupby(['Departure', 'Arrival', 'month']).mean()).reset_index()
print(X_groupby.columns)

merge_transform = MergeTransformer(X_ext=X_groupby, 
                                   cols_to_keep=['Departure', 'Arrival', 'month', 'Passengers'],
                                   cols_to_rename={'Departure': 'AirPort', 'Passengers': 'monthly_route_mean'},
                                   how='left',
                                   on=['AirPort', 'Arrival', 'month'])
ext_data = merge_transform.fit_transform(ext_data)
ext_data.isna().sum()

<ipython-input-3-436ef0b91cdd>:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week


Index(['DateOfDeparture', 'AirPort', 'n_days', 'oil_stock_price',
       'oil_stock_volume', 'AAL_stock_price', 'AAL_stock_volume',
       'SP_stock_price', 'SP_stock_volume', 'day_mean', 'week_mean',
       'month_mean', 'day_nb_mean', 'route_mean', 'Arrival', 'distance',
       'Total', 'Flights', 'Booths', 'Mean per flight', 'year', 'month', 'day',
       'weekday', 'week', 'day_nb'],
      dtype='object')
Index(['Departure', 'Arrival', 'month', 'WeeksToDeparture', 'std_wtd',
       'Passengers', 'year', 'day', 'weekday', 'week', 'n_days', 'day_nb'],
      dtype='object')


In [50]:
ext_data = pd.read_csv('../data/external_data.csv')
print(ext_data.shape)
print(ext_data.isna().sum())
ext_data.dropna(inplace=True, axis=1)
print(ext_data.shape)

date_encoder = FunctionTransformer(_encode_dates)
ext_data = date_encoder.fit_transform(ext_data)

ext_data.drop('Unnamed: 0', axis=1, inplace=True)
print(ext_data.columns)

X_groupby = pd.DataFrame(Xy_merged.groupby(['Departure', 'Arrival', 'weekday']).mean()).reset_index()
print(X_groupby.head())

merge_transform = MergeTransformer(X_ext=X_groupby, 
                                   cols_to_keep=['Departure', 'Arrival', 'weekday', 'Passengers'],
                                   cols_to_rename={'Departure': 'AirPort', 'Passengers': 'weekday_route_mean'},
                                   how='left',
                                   on=['AirPort', 'Arrival', 'weekday'])
ext_data = merge_transform.fit_transform(ext_data)
print(ext_data.shape)
ext_data.isna().sum()



(70656, 22)
Unnamed: 0          0
DateOfDeparture     0
AirPort             0
n_days              0
day_nb              0
oil_stock_price     0
oil_stock_volume    0
AAL_stock_price     0
AAL_stock_volume    0
SP_stock_price      0
SP_stock_volume     0
day_mean            0
week_mean           0
month_mean          0
day_nb_mean         0
route_mean          0
Arrival             0
distance            0
Total               0
Flights             0
Booths              0
Mean per flight     0
dtype: int64
(70656, 22)


<ipython-input-3-436ef0b91cdd>:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  X_encoded.loc[:, 'week'] = X_encoded['DateOfDeparture'].dt.week


Index(['DateOfDeparture', 'AirPort', 'n_days', 'oil_stock_price',
       'oil_stock_volume', 'AAL_stock_price', 'AAL_stock_volume',
       'SP_stock_price', 'SP_stock_volume', 'day_mean', 'week_mean',
       'month_mean', 'day_nb_mean', 'route_mean', 'Arrival', 'distance',
       'Total', 'Flights', 'Booths', 'Mean per flight', 'year', 'month', 'day',
       'weekday', 'week', 'day_nb'],
      dtype='object')
  Departure Arrival  weekday  WeeksToDeparture   std_wtd  Passengers  \
0       ATL     BOS      0.0         10.649103  8.156233   11.306579   
1       ATL     BOS      1.0          9.955832  7.838169   10.871567   
2       ATL     BOS      2.0          9.379901  7.345966   11.418848   
3       ATL     BOS      3.0         10.268807  8.151156   11.592279   
4       ATL     BOS      4.0         10.969277  8.483814   11.239484   

          year     month        day       week        n_days      day_nb  
0  2011.764706  7.470588  18.705882  31.470588  15468.647059  214.941176  
1  2

DateOfDeparture          1
AirPort                  1
n_days                   1
oil_stock_price          1
oil_stock_volume         1
AAL_stock_price          1
AAL_stock_volume         1
SP_stock_price           1
SP_stock_volume          1
day_mean                 1
week_mean                1
month_mean               1
day_nb_mean              1
route_mean               1
Arrival                  1
distance                 1
Total                    1
Flights                  1
Booths                   1
Mean per flight          1
year                     1
month                    1
day                      1
weekday                  1
week                     1
day_nb                   1
weekday_route_mean    1105
dtype: int64

In [54]:
ext_data_droped_last = ext_data.drop(70656)

In [55]:
ext_data_droped_last.isna().sum()

DateOfDeparture          0
AirPort                  0
n_days                   0
oil_stock_price          0
oil_stock_volume         0
AAL_stock_price          0
AAL_stock_volume         0
SP_stock_price           0
SP_stock_volume          0
day_mean                 0
week_mean                0
month_mean               0
day_nb_mean              0
route_mean               0
Arrival                  0
distance                 0
Total                    0
Flights                  0
Booths                   0
Mean per flight          0
year                     0
month                    0
day                      0
weekday                  0
week                     0
day_nb                   0
weekday_route_mean    1104
dtype: int64

In [61]:
ext_data_droped_last.drop(['year', 'month', 'day', 'weekday', 'week', 'day_nb'], axis=1, inplace=True)

KeyError: "['year' 'month' 'day' 'weekday' 'week' 'day_nb'] not found in axis"

In [62]:
ext_data_droped_last

,DateOfDeparture,AirPort,n_days,oil_stock_price,oil_stock_volume,AAL_stock_price,AAL_stock_volume,SP_stock_price,SP_stock_volume,day_mean,...,month_mean,day_nb_mean,route_mean,Arrival,distance,Total,Flights,Booths,Mean per flight,weekday_route_mean
0,2011-01-09,ATL,14983.0,88.93,282342.0,5.29,6677100.0,1204.420044,4.780410e+09,11.358502,...,11.163944,10.7111,11.554774,DFW,1177.006614,5061.0,51.0,244.0,99.24,11.378641
1,2011-01-09,ATL,14983.0,88.93,282342.0,5.29,6677100.0,1204.420044,4.780410e+09,11.358502,...,11.163944,10.7111,10.365239,DEN,1930.017732,803.0,7.0,31.0,114.71,10.586533
2,2011-01-09,ATL,14983.0,88.93,282342.0,5.29,6677100.0,1204.420044,4.780410e+09,11.358502,...,11.163944,10.7111,11.288425,LAX,3132.519497,22513.0,105.0,928.0,214.41,11.305570
3,2011-01-09,ATL,14983.0,88.93,282342.0,5.29,6677100.0,1204.420044,4.780410e+09,11.358502,...,11.163944,10.7111,11.300291,ORD,974.957144,12436.0,62.0,412.0,200.58,10.889180
4,2011-01-09,ATL,14983.0,88.93,282342.0,5.29,6677100.0,1204.420044,4.780410e+09,11.358502,...,11.163944,10.7111,10.462382,SFO,3442.220394,11567.0,49.0,518.0,236.06,10.660753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70651,2013-05-03,LGA,15828.0,90.82,209874.0,14.05,7676100.0,1539.790039,3.610690e+09,11.080302,...,10.946436,10.6809,10.773446,CLT,875.196007,2583.0,17.0,80.0,151.94,10.957733
70652,2013-05-03,LGA,15828.0,90.82,209874.0,14.05,7676100.0,1539.790039,3.610690e+09,11.080302,...,10.946436,10.6809,10.590409,DTW,807.104383,2938.0,20.0,144.0,146.90,10.744285
70653,2013-05-03,LGA,15828.0,90.82,209874.0,14.05,7676100.0,1539.790039,3.610690e+09,11.080302,...,10.946436,10.6809,10.972960,MIA,1764.808838,21791.0,173.0,832.0,125.96,11.319265
70654,2013-05-03,LGA,15828.0,90.82,209874.0,14.05,7676100.0,1539.790039,3.610690e+09,11.080302,...,10.946436,10.6809,11.168998,BOS,297.068044,2501.0,22.0,129.0,113.68,11.140082


In [63]:
ext_data_droped_last.to_csv('../submissions/ext_data_store/external_data_routeweekday.csv')

In [29]:
gewkgweijfew

NameError: name 'gewkgweijfew' is not defined

## Encode Dates and RandomForestRegressor

In [ ]:
X_test.drop(['DateOfDeparture', 'weekday', 'month', 'week'], axis=1, inplace=True)
X_train.drop(['DateOfDeparture', 'weekday', 'month', 'week'], axis=1, inplace=True)

In [ ]:
X_train_lol = X_train.drop(['Departure', 'Arrival'], axis=1)
X_test_lol = X_test.drop(['Departure', 'Arrival'], axis=1)

In [ ]:
X_train_lol.head()
X_test_lol.head()

In [ ]:
grid_params = {
    'n_estimators': [100],
    'min_samples_split': [0.001, 0.005, 0.01],
#     'oob_score': [True]
#     'max_features': [0.5, 0.75]
}

gs = GridSearchCV(estimator=RandomForestRegressor(),
                  param_grid=grid_params,
                  n_jobs=-1,
                  cv=5,
                  verbose=0)

gs.fit(X_train_lol, y_train)        
print(gs.best_score_)
print(gs.best_params_) 

In [ ]:
print("Training RMSE: ", mean_squared_error(y_train, gs.predict(X_train_lol), squared=False))
print("Testing RMSE: ", mean_squared_error(y_test, gs.predict(X_test_lol), squared=False))

In [ ]:
gs.score(X_test_lol, y_test)

In [ ]:
os.environ['PATH'] = os.environ['PATH']+';'+os.environ['CONDA_PREFIX']+r"\Library\bin\graphviz"
dot_data = StringIO()
export_graphviz(gs.best_estimator_[0], out_file=dot_data)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue()) 
Image(graph.create_png())